In [3]:
# @title 1. Cài đặt Thư viện (Optimized)
import sys
import subprocess
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "transformers", "sentence-transformers", "tokenizers"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "transformers==4.44.2", "tokenizers==0.15.2"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "accelerate==0.34.2", "bitsandbytes==0.43.1", "sentencepiece"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "fastapi", "uvicorn", "python-multipart", "pyngrok", "nest_asyncio"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gTTS", "librosa", "soundfile", "edge-tts"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "faster-whisper"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "triton==2.1.0"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "chromadb", "llama-index", "llama-index-vector-stores-chroma", "llama-index-embeddings-langchain", "langchain-community", "sentence-transformers==2.7.0", "Pillow"])
print("✅ Đã cài đặt xong thư viện (Phiên bản tối ưu VRAM).")


✅ Đã cài đặt xong thư viện (Phiên bản tối ưu VRAM).


In [4]:
# @title 2. Khởi tạo Model (Whisper chạy CPU để cứu GPU)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor, LlavaNextForConditionalGeneration, BitsAndBytesConfig
from faster_whisper import WhisperModel
import os
import gc

# 1. Dọn dẹp sạch sẽ bộ nhớ trước khi load
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
gc.collect()

# --- CẤU HÌNH ---
CHAT_MODEL_ID = "unsloth/Llama-3.2-3B-Instruct"
VLM_MODEL_ID = "llava-hf/llava-1.5-7b-hf"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"▶ Đang chạy trên thiết bị chính: {device.upper()}")

# Cấu hình 4-bit (Chìa khóa để chạy trên T4)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# ------------------------------------------------------------------
# A. Load Chat Model (Llama 3.2 3B - 4bit) -> GPU
# ------------------------------------------------------------------
print("⏳ [1/3] Loading Chat Model (GPU 4-bit)...")
if 'chat_tokenizer' in globals() and chat_tokenizer is not None and 'chat_model' in globals() and chat_model is not None:
    print("   ✔ Chat Model already loaded.")
else:
    chat_tokenizer = AutoTokenizer.from_pretrained(CHAT_MODEL_ID)
    try:
        chat_model = AutoModelForCausalLM.from_pretrained(
            CHAT_MODEL_ID,
            quantization_config=bnb_config,
            device_map="auto"
        )
        print("   ✔ Chat Model OK.")
    except Exception:
        chat_model = AutoModelForCausalLM.from_pretrained(
            CHAT_MODEL_ID,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        print("   ✔ Chat Model OK (fp16 fallback).")

# ------------------------------------------------------------------
# B. Load Vision Model (Llava 7B - 4bit) -> GPU
# ------------------------------------------------------------------
print("⏳ [2/3] Loading Vision Model (GPU 4-bit)...")
from transformers import LlavaForConditionalGeneration
if 'vlm_processor' in globals() and vlm_processor is not None and 'vlm_model' in globals() and vlm_model is not None:
    print("   ✔ Vision Model already loaded.")
else:
    vlm_processor = AutoProcessor.from_pretrained(VLM_MODEL_ID)
    try:
        vlm_model = LlavaForConditionalGeneration.from_pretrained(
            VLM_MODEL_ID,
            quantization_config=bnb_config,
            device_map="auto"
        )
        print("   ✔ Vision Model OK.")
    except Exception:
        vlm_model = LlavaForConditionalGeneration.from_pretrained(
            VLM_MODEL_ID,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        print("   ✔ Vision Model OK (fp16 fallback).")

# ------------------------------------------------------------------
# C. Load Whisper (Faster-Whisper) -> CPU (ĐỂ TRÁNH OOM)
# ------------------------------------------------------------------
print("⏳ [3/3] Loading Faster-Whisper (CPU Int8)...")
if 'stt_model' in globals() and stt_model is not None:
    print("   ✔ Whisper Model already loaded.")
else:
    stt_model = WhisperModel("medium", device="cpu", compute_type="int8")
    print("   ✔ Whisper Model OK.")

# ------------------------------------------------------------------
print("\n📊 Trạng thái VRAM sau khi load:")
print(torch.cuda.memory_summary(abbreviated=True))
print("🚀 TẤT CẢ MODEL ĐÃ SẴN SÀNG (Llama+Llava on GPU, Whisper on CPU)!")


▶ Đang chạy trên thiết bị chính: CUDA
⏳ [1/3] Loading Chat Model (GPU 4-bit)...


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

   ✔ Whisper Model OK.

📊 Trạng thái VRAM sau khi load:
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  13561 MiB |  13561 MiB |  33199 MiB |  19637 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  13561 MiB |  13561 MiB |  33199 MiB |  19637 MiB |
|---------------------------------------------------------------------------|
| Requested memory      |  13559 MiB |  13559 MiB |  33196 MiB |  19637 MiB |
|-------

In [5]:
# @title 3. Kết nối Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

Mounted at /content/drive


In [6]:
# @title 4. Cài đặt RAG
import chromadb
from langchain_community.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
try:
    from sentence_transformers import CrossEncoder
except Exception:
    CrossEncoder = None
DB_ALL_PATH = "/content/drive/MyDrive/DoctorAI - Trợ lý Sức khoẻ Cá nhân Thông minh/data/DB_ALL"
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="bkai-foundation-models/vietnamese-bi-encoder", model_kwargs={'device': device}))
chroma_client = chromadb.PersistentClient(path=DB_ALL_PATH)
chroma_collection = chroma_client.get_or_create_collection("KienThucYKhoa")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, embed_model=embed_model, storage_context=storage_context)
retriever = index.as_retriever(similarity_top_k=10)
reranker = None
if CrossEncoder is not None:
    try:
        reranker = CrossEncoder("Alibaba-NLP/gte-multilingual-reranker-base", trust_remote_code=True)
    except Exception:
        reranker = None


/tmp/ipython-input-383238384.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="bkai-foundation-models/vietnamese-bi-encoder", model_kwargs={'device': device}))


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/612M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [9]:
# @title 5. cấu hình ngrok
import base64
import uuid
import os
import json
import asyncio
import subprocess
import threading
import time
import socket
from io import BytesIO
from typing import List, Optional

USE_NGROK = True
def pick_free_port(preferred: int = 8000) -> int:
    for p in [preferred] + list(range(preferred + 1, preferred + 50)):
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        try:
            s.bind(("0.0.0.0", p))
            s.close()
            return p
        except OSError:
            s.close()
            continue
    return preferred
PORT = pick_free_port(8000)
NGROK_AUTH_TOKEN = None
if USE_NGROK:
    NGROK_AUTH_TOKEN = os.environ.get("NGROK_AUTH_TOKEN") or os.environ.get("NGROK_TOKEN")
    if not NGROK_AUTH_TOKEN:
        try:
            from getpass import getpass
            NGROK_AUTH_TOKEN = getpass("Nhập NGROK_AUTH_TOKEN: ")
            if NGROK_AUTH_TOKEN:
                os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN
        except Exception:
            NGROK_AUTH_TOKEN = None


Nhập NGROK_AUTH_TOKEN: ··········


In [ ]:
# @title 6. Khởi chạy Server API (VRAM Safe Mode)
from fastapi import FastAPI, File, UploadFile, Form, Header
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse, JSONResponse
from pydantic import BaseModel
from typing import List, Optional
import nest_asyncio
from pyngrok import ngrok
from PIL import Image
import librosa
import soundfile as sf
import numpy as np
from gtts import gTTS
import threading
import time
import socket

# Patch async cho Colab
try:
    nest_asyncio.apply()
except Exception:
    pass

app = FastAPI(title="Medical Consultation GPU API", version="2.1.0")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# ==============================
# 🔶 DATA MODELS
# ==============================
class ChatMessage(BaseModel):
    role: str
    content: str

class ChatRequest(BaseModel):
    model: Optional[str] = "llama-3.2-3b"
    messages: List[ChatMessage]
    temperature: Optional[float] = 0.7
    max_tokens: Optional[int] = 512
    mode: Optional[str] = "pro"

class VisionMultiRequest(BaseModel):
    text: str
    images_base64: List[str]
    temperature: Optional[float] = 0.2
    max_tokens: Optional[int] = 256

class TTSRequest(BaseModel):
    text: str
    lang: Optional[str] = "vi"
class AutoTitleRequest(BaseModel):
    messages: Optional[List[ChatMessage]] = None
    user_text: Optional[str] = None
    assistant_text: Optional[str] = None
    max_tokens: Optional[int] = 24

# ==============================
# 🔷 1. CHAT API
# ==============================
@app.post("/v1/chat/completions")
async def chat_completions(req: ChatRequest, x_mode: Optional[str] = Header(None)):
    try:
        msgs = req.messages or []
        question = ""
        for m in reversed(msgs):
            if m.role.lower() == "user":
                question = m.content
                break
        if not question and msgs:
            question = msgs[-1].content
        try:
            if question:
                print(f"[USER] { question }")
        except Exception:
            pass
        mode = (x_mode or req.mode or "pro").lower()
        classify_prompt = "trả lời ngắn gọn là có hay không và không giải thích gì thêm: câu hỏi sau đây có liên quan y tế không: " + question
        cls_text_tmpl = chat_tokenizer.apply_chat_template([{"role": "user", "content": classify_prompt}], tokenize=False, add_generation_prompt=True)
        cls_inputs = chat_tokenizer(cls_text_tmpl, return_tensors="pt").to("cuda")
        with torch.no_grad():
            cls_out = chat_model.generate(**cls_inputs, max_new_tokens=8, temperature=0, do_sample=False, pad_token_id=chat_tokenizer.eos_token_id)
        cls_resp = chat_tokenizer.decode(cls_out[0][cls_inputs.input_ids.shape[-1]:], skip_special_tokens=True)
        del cls_inputs, cls_out
        torch.cuda.empty_cache()
        if "không" in cls_resp.lower():
            response_text = "Câu hỏi của bạn không liên quan đến y tế. Vui lòng đặt câu hỏi khác."
            return {
                "id": f"chatcmpl-{uuid.uuid4()}",
                "object": "chat.completion",
                "created": int(time.time()),
                "choices": [{"index": 0, "message": {"role": "assistant", "content": response_text}, "finish_reason": "stop"}],
                "mode": mode
            }
        if mode == "pro":
            nodes = retriever.retrieve(question)
            context_passages = [n.node.get_content() for n in nodes]
            ranked = context_passages
            try:
                if reranker is not None:
                    query_passage_pairs = [[question, p] for p in context_passages]
                    scores = reranker.predict(query_passage_pairs)
                    ranked = [p for _, p in sorted(zip(scores, context_passages), key=lambda x: x[0], reverse=True)]
            except Exception:
                ranked = context_passages
            top_k = min(3, len(ranked))
            selected = ranked[:top_k]
            ctx = "Đây là câu hỏi của người dùng:\n" + question + "\n\n"
            ctx += "Dưới đây là các đoạn thông tin liên quan:\n"
            for i, p in enumerate(selected):
                ctx += "\n[Đoạn " + str(i + 1) + "]:\n" + p + "\n"
            doctor_prompt = "Bạn là bác sỹ tư vấn y tế, không kê đơn thuốc, không chẩn đoán thay thế chuyên môn. Trả lời tiếng Việt, ngắn gọn, rõ ràng, ưu tiên an toàn và khuyến cáo gặp bác sỹ khi cần."
            input_text = chat_tokenizer.apply_chat_template(
                [{"role": "system", "content": doctor_prompt}, {"role": "user", "content": ctx}],
                tokenize=False,
                add_generation_prompt=True
            )
            inputs = chat_tokenizer(input_text, return_tensors="pt").to("cuda")
            with torch.no_grad():
                output = chat_model.generate(
                    **inputs,
                    max_new_tokens=req.max_tokens,
                    temperature=req.temperature,
                    do_sample=True if req.temperature > 0 else False,
                    pad_token_id=chat_tokenizer.eos_token_id
                )
            response_text = chat_tokenizer.decode(output[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)
            del inputs, output
            torch.cuda.empty_cache()
        else:
            doctor_prompt = "Bạn là bác sỹ tư vấn y tế, không kê đơn thuốc, không chẩn đoán thay thế chuyên môn. Trả lời tiếng Việt, ngắn gọn, rõ ràng, ưu tiên an toàn và khuyến cáo gặp bác sỹ khi cần."
            input_text = chat_tokenizer.apply_chat_template(
                [{"role": "system", "content": doctor_prompt}, {"role": "user", "content": question}],
                tokenize=False,
                add_generation_prompt=True
            )
            inputs = chat_tokenizer(input_text, return_tensors="pt").to("cuda")
            with torch.no_grad():
                output = chat_model.generate(
                    **inputs,
                    max_new_tokens=req.max_tokens,
                    temperature=req.temperature,
                    do_sample=True if req.temperature > 0 else False,
                    pad_token_id=chat_tokenizer.eos_token_id
                )
            response_text = chat_tokenizer.decode(output[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)
            del inputs, output
            torch.cuda.empty_cache()
        return {
            "id": f"chatcmpl-{uuid.uuid4()}",
            "object": "chat.completion",
            "created": int(time.time()),
            "choices": [{"index": 0, "message": {"role": "assistant", "content": response_text}, "finish_reason": "stop"}],
            "mode": mode
        }
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

@app.post("/v1/chat")
async def chat_simple(req: dict, x_mode: Optional[str] = Header(None)):
    # Proxy đơn giản cho chat
    msgs = req.get("messages", [])
    if not msgs: return {"reply": ""}

    question = ""
    for m in reversed(msgs):
        if m.get("role", "").lower() == "user":
            question = m.get("content", "")
            break
    if not question and msgs:
        question = msgs[-1].get("content", "")
    try:
        if question:
            print(f"[USER] { question }")
    except Exception:
        pass
    mode = (x_mode or req.get("mode") or "pro").lower()
    classify_prompt = "trả lời ngắn gọn là có hay không và không giải thích gì thêm: câu hỏi sau đây có liên quan y tế không: " + question
    cls_text_tmpl = chat_tokenizer.apply_chat_template([{"role": "user", "content": classify_prompt}], tokenize=False, add_generation_prompt=True)
    cls_inputs = chat_tokenizer(cls_text_tmpl, return_tensors="pt").to("cuda")
    with torch.no_grad():
        cls_out = chat_model.generate(**cls_inputs, max_new_tokens=8, temperature=0, do_sample=False, pad_token_id=chat_tokenizer.eos_token_id)
    cls_resp = chat_tokenizer.decode(cls_out[0][cls_inputs.input_ids.shape[-1]:], skip_special_tokens=True)
    del cls_inputs, cls_out
    torch.cuda.empty_cache()
    if "không" in cls_resp.lower():
        return {"reply": "Câu hỏi của bạn không liên quan đến y tế. Vui lòng đặt câu hỏi khác."}
    if mode == "pro":
        nodes = retriever.retrieve(question)
        context_passages = [n.node.get_content() for n in nodes]
        ranked = context_passages
        try:
            if reranker is not None:
                query_passage_pairs = [[question, p] for p in context_passages]
                scores = reranker.predict(query_passage_pairs)
                ranked = [p for _, p in sorted(zip(scores, context_passages), key=lambda x: x[0], reverse=True)]
        except Exception:
            ranked = context_passages
        top_k = min(3, len(ranked))
        selected = ranked[:top_k]
        ctx = "Đây là câu hỏi của người dùng:\n" + question + "\n\n"
        ctx += "Dưới đây là các đoạn thông tin liên quan:\n"
        for i, p in enumerate(selected):
            ctx += "\n[Đoạn " + str(i + 1) + "]:\n" + p + "\n"
        doctor_prompt = "Bạn là bác sỹ tư vấn y tế, không kê đơn thuốc, không chẩn đoán thay thế chuyên môn. Trả lời tiếng Việt, ngắn gọn, rõ ràng, ưu tiên an toàn và khuyến cáo gặp bác sỹ khi cần."
        input_text = chat_tokenizer.apply_chat_template(
            [{"role": "system", "content": doctor_prompt}, {"role": "user", "content": ctx}],
            tokenize=False,
            add_generation_prompt=True
        )
    else:
        doctor_prompt = "Bạn là bác sỹ tư vấn y tế, không kê đơn thuốc, không chẩn đoán thay thế chuyên môn. Trả lời tiếng Việt, ngắn gọn, rõ ràng, ưu tiên an toàn và khuyến cáo gặp bác sỹ khi cần."
        input_text = chat_tokenizer.apply_chat_template(
            [{"role": "system", "content": doctor_prompt}, {"role": "user", "content": question}],
            tokenize=False,
            add_generation_prompt=True
        )
    inputs = chat_tokenizer(input_text, return_tensors="pt").to("cuda")
    with torch.no_grad():
        output = chat_model.generate(**inputs, max_new_tokens=256, temperature=0.7, do_sample=True, pad_token_id=chat_tokenizer.eos_token_id)
    text = chat_tokenizer.decode(output[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)
    del inputs, output
    torch.cuda.empty_cache()
    return {"reply": text, "mode": mode}

@app.post("/v1/auto-title")
async def auto_title(req: AutoTitleRequest):
    try:
        user_text = req.user_text or ""
        assistant_text = req.assistant_text or ""
        if (not user_text or not assistant_text) and req.messages:
            for m in reversed(req.messages):
                if m.role.lower() == "assistant" and not assistant_text:
                    assistant_text = m.content
                elif m.role.lower() == "user" and not user_text:
                    user_text = m.content
                if user_text and assistant_text:
                    break
        if not user_text:
            user_text = ""
        if not assistant_text:
            assistant_text = ""
        prompt = "Tạo tiêu đề ngắn gọn bằng tiếng Việt (4-8 từ) cho hội thoại sau. Không thêm dấu chấm cuối câu, không ngoặc kép.\nNgười dùng: " + user_text + "\nTrợ lý: " + assistant_text
        input_text = chat_tokenizer.apply_chat_template(
            [{"role": "system", "content": "Bạn là trợ lý đặt tiêu đề."}, {"role": "user", "content": prompt}],
            tokenize=False,
            add_generation_prompt=True
        )
        inputs = chat_tokenizer(input_text, return_tensors="pt").to("cuda")
        with torch.no_grad():
            output = chat_model.generate(
                **inputs,
                max_new_tokens=req.max_tokens or 24,
                temperature=0.2,
                do_sample=False,
                pad_token_id=chat_tokenizer.eos_token_id
            )
        title = chat_tokenizer.decode(output[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True).strip()
        try:
            import re
            title = re.sub(r'[\r\n]+', ' ', title)
            title = re.sub(r'[*_`#]+', '', title)
            title = re.sub(r'\s+', ' ', title)
            title = title[:60]
        except Exception:
            title = title[:60]
        del inputs, output
        torch.cuda.empty_cache()
        return {"title": title, "mode": "gpu"}
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

# ==============================
# 🔶 2. VISION API
# ==============================
@app.post("/v1/vision-multi")
async def vision_multi_api(req: VisionMultiRequest):
    try:
        full_response = []
        # Giới hạn xử lý 2 ảnh để tránh tràn RAM
        for idx, b64_str in enumerate(req.images_base64[:2]):
            image_bytes = base64.b64decode(b64_str)
            image = Image.open(BytesIO(image_bytes)).convert("RGB")

            prompt = f"USER: <image>\n{req.text}\nASSISTANT:"
            inputs = vlm_processor(text=prompt, images=image, return_tensors="pt").to("cuda")

            with torch.no_grad():
                output = vlm_model.generate(**inputs, max_new_tokens=req.max_tokens)

            resp = vlm_processor.decode(output[0], skip_special_tokens=True).split("ASSISTANT:")[-1].strip()
            full_response.append(f"[Ảnh {idx+1}]: {resp}")

            del inputs, output, image
            torch.cuda.empty_cache()

        return {"success": True, "response": "\n".join(full_response)}
    except Exception as e:
        return {"success": False, "error": str(e)}

# ==============================
# 🔷 3. TTS STREAMING (gTTS)
# ==============================
@app.post("/v1/tts/stream")
async def tts_stream_api(req: TTSRequest):
    async def generate_audio_stream():
        temp_filename = f"/content/tts_{uuid.uuid4().hex}.mp3"
        try:
            tts = gTTS(text=req.text, lang=req.lang)
            tts.save(temp_filename)
            chunk_size = 32 * 1024
            chunk_id = 0
            with open(temp_filename, "rb") as f:
                while True:
                    data = f.read(chunk_size)
                    if not data: break
                    b64_data = base64.b64encode(data).decode("utf-8")
                    yield json.dumps({"chunk_id": chunk_id, "audio_base64": b64_data}) + "\n"
                    chunk_id += 1
                    await asyncio.sleep(0.01)
            if os.path.exists(temp_filename): os.remove(temp_filename)
        except Exception as e:
            yield json.dumps({"error": str(e)}) + "\n"

    return StreamingResponse(generate_audio_stream(), media_type="application/x-ndjson")

# ==============================
# 🔶 4. STT STREAMING (Faster-Whisper)
# ==============================
@app.post("/v1/stt/stream")
async def stt_stream_api(file: UploadFile = File(...)):
    async def transcribe_stream():
        temp_filename = f"/content/upload_{uuid.uuid4().hex}.wav"
        try:
            with open(temp_filename, "wb") as f:
                f.write(await file.read())

            # Faster-whisper trả về generator
            segments, info = stt_model.transcribe(temp_filename, beam_size=5, language="vi")

            for segment in segments:
                payload = json.dumps({
                    "text": segment.text,
                    "start": segment.start,
                    "end": segment.end,
                    "partial": segment.text
                })
                yield payload + "\n"
                await asyncio.sleep(0.01)

            if os.path.exists(temp_filename): os.remove(temp_filename)
        except Exception as e:
            yield json.dumps({"error": str(e)}) + "\n"

    return StreamingResponse(transcribe_stream(), media_type="application/x-ndjson")

# ==============================
# 🚀 STARTUP
# ==============================
@app.get("/gpu/metrics")
async def gpu_metrics():
    # Lấy thông số GPU thật
    try:
        r = subprocess.run(["nvidia-smi", "--query-gpu=utilization.gpu,memory.used,memory.total", "--format=csv,noheader,nounits"], capture_output=True, text=True)
        u, m_used, m_total = r.stdout.strip().split(",")
        return {"gpu_utilization": float(u), "mem_used": float(m_used), "mem_total": float(m_total)}
    except:
        return {}

@app.get("/health")
async def health():
    try:
        ok_chat = chat_model is not None
        ok_tokenizer = chat_tokenizer is not None
        ok_vlm = vlm_model is not None and vlm_processor is not None
        ok_stt = stt_model is not None
        return {"status": "ok", "chat": ok_chat, "tokenizer": ok_tokenizer, "vlm": ok_vlm, "stt": ok_stt}
    except Exception:
        return {"status": "degraded"}

@app.get("/")
async def root():
    return {"status": "ok"}

print("--- KHỞI ĐỘNG SERVER (VRAM OPTIMIZED) ---")

def find_free_port() -> int:
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

def is_port_available(p: int) -> bool:
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind(('0.0.0.0', p))
        s.close()
        return True
    except OSError:
        return False

def run_uvicorn():
    import uvicorn
    import asyncio
    try:
        config = uvicorn.Config(app, host="0.0.0.0", port=PORT, loop="asyncio", lifespan="on")
        server = uvicorn.Server(config)
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(server.serve())
    except Exception as e:
        print("UVicorn start error:", str(e))

if not is_port_available(PORT):
    new_port = find_free_port()
    print(f"⚠️ Port {PORT} đang được sử dụng, chuyển sang port {new_port}")
    PORT = new_port
else:
    print(f"✅ Sử dụng port {PORT}")

thread = threading.Thread(target=run_uvicorn, daemon=True)
thread.start()

time.sleep(1.0)

if USE_NGROK and NGROK_AUTH_TOKEN:
    try:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        ngrok.kill()
        public_url = ngrok.connect(PORT)
        print(f"✅ Public URL: {public_url.public_url}")
        print(f"🔗 Health: {public_url.public_url}/health")
    except Exception as e:
        print(f"❌ Ngrok error: {e}")
else:
    print("⚠️ Chạy Localhost")

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    pass


--- KHỞI ĐỘNG SERVER (VRAM OPTIMIZED) ---
⚠️ Port 41605 đang được sử dụng, chuyển sang port 58533


INFO:     Started server process [281]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:58533 (Press CTRL+C to quit)


✅ Public URL: https://miyoko-trichomonadal-reconditely.ngrok-free.dev
🔗 Health: https://miyoko-trichomonadal-reconditely.ngrok-free.dev/health
